In [4]:
pip install cvzone

Note: you may need to restart the kernel to use updated packages.


In [85]:
# Load the video from the system and display it

import cv2 
import cvzone
from cvzone.ColorModule import ColorFinder
import math
import numpy as np

# Global constant for video input file
global FILENAME
FILENAME = 'Videos\\vid (5).mp4'

cap = cv2.VideoCapture(FILENAME)

while True :
    # Grab the image 
    success, img = cap.read()

    try: 
        # Shrinking image size to fit the screen
        img = cv2.resize(img, (224, 224), fx=0, fy=0, interpolation = None)
    except:
        break
   
    # dispay image for specified milliseconds
    cv2.imshow('Image', img)
    cv2.waitKey(50)

# close all the windows after displaying the image
cv2.destroyAllWindows()

In [86]:
# Identify the ball feature from the image

# We will be needing this code for the first time only to identify the ball from the image
'''
# True: Running in debug mode, trying to find the color of the ball from the image
myColorFinder = ColorFinder(True)
hsvVals = {'hmin': 8, 'smin': 96, 'vmin':115 , 'hmax': 14, 'smax': 255, 'vmax': 255}

while True :
    # Reading the ball image to identify the basketball color
    img = cv2.imread('Ball.png')
    
    # Crop the image to get more clearity of the ball 
    img = img[0 : 900, : ]
    
    # Find the color of the ball by manipulating HSL (hue, saturation, lightness) 
    imgColor, mask = myColorFinder.update(img, hsvVals)
    
    try:
        img = cv2.resize(img, (224, 224), fx=0, fy=0, interpolation=None)
    except:
        break
    
    # Reflect the changes in HSL in the image to better manipulate the image
    cv2.imshow('ImageColor', imgColor)
    cv2.waitKey(50)

# close all the windows after displaying the image
cv2.destroyAllWindows()
'''

"\n# True: Running in debug mode, trying to find the color of the ball from the image\nmyColorFinder = ColorFinder(True)\nhsvVals = {'hmin': 8, 'smin': 96, 'vmin':115 , 'hmax': 14, 'smax': 255, 'vmax': 255}\n\nwhile True :\n    # Reading the ball image to identify the basketball color\n    img = cv2.imread('Ball.png')\n    \n    # Crop the image to get more clearity of the ball \n    img = img[0 : 900, : ]\n    \n    # Find the color of the ball by manipulating HSL (hue, saturation, lightness) \n    imgColor, mask = myColorFinder.update(img, hsvVals)\n    \n    try:\n        img = cv2.resize(img, (224, 224), fx=0, fy=0, interpolation=None)\n    except:\n        break\n    \n    # Reflect the changes in HSL in the image to better manipulate the image\n    cv2.imshow('ImageColor', imgColor)\n    cv2.waitKey(50)\n\n# close all the windows after displaying the image\ncv2.destroyAllWindows()\n"

In [87]:
# Detecting the basketball after identifying it from the image

while True :
    img = cv2.imread('Ball.png')

    try:
        # Find the colour of the ball HSL (for hue, saturation, lightness) 
        imgColor, mask = myColorFinder.update(img, hsvVals)
    except:
        break

    img = cv2.resize(imgColor, (0, 0), fx=0.7, fy=0.7, interpolation=None)
    cv2.imshow('ImageColor', imgColor)
    cv2.waitKey(50)

    if cv2.waitKey(5000):
        break

cv2.destroyAllWindows()

In [88]:
# Reflecting the ball detection in the video

cap = cv2.VideoCapture(FILENAME)

while True :
    success, img = cap.read()
    
    try:
        # Find the colour of the ball HSL (for hue, saturation, lightness) 
        imgColor, mask = myColorFinder.update(img, hsvVals)
    except:
        break

    img = cv2.resize(imgColor, (0, 0), fx=0.7, fy=0.7, interpolation=None)
    cv2.imshow('imageColor', imgColor)
    cv2.waitKey(25)
    
    if cv2.waitKey(25) & 0XFF == 27:
        break

cv2.destroyAllWindows()

In [89]:
# Find the ball using contour points

cap = cv2.VideoCapture(FILENAME)

# Create the color finder object :
myColorFinder = ColorFinder(False)
hsvVals = {'hmin': 0, 'smin': 105, 'vmin':0 , 'hmax': 15, 'smax': 255, 'vmax': 255}

while True :
    
    # Grab the video
    success, img = cap.read()
    
    try:
        # Find the colour of the ball HSL (for hue, saturation, lightness) 
        imgColor, mask = myColorFinder.update(img, hsvVals)
        # Find the location of ball 
        imgContours, contours = cvzone.findContours(img, mask, minArea=500)
    except:
        break

    if contours:
        # Choose the biggest contours(the first element) and its center
        cx, cy = contours[0]['center']
        # print(cx, cy)
        # cv2.circle(img, center, radius , color, thickness)
        cv2.circle(imgContours, (cx, cy), 5, (0, 255, 0), cv2.FILLED)
    
    try:
        img = cv2.resize(imgContours, (0, 0), fx=0.7, fy=0.7, interpolation = None)
    except:
        break
   
    cv2.imshow('imgColor', imgContours)

    if cv2.waitKey(50) & 0XFF== 27:
        break 

# close all the windows after displaying the image
cv2.destroyAllWindows()

In [90]:
# Plot all the contour points in the final image tracing the basketball and then connecting them to form a line

cap = cv2.VideoCapture(FILENAME)

# Create the color finder object :
myColorFinder = ColorFinder(False)
hsvVals ={'hmin': 8, 'smin': 96, 'vmin':115 , 'hmax': 14, 'smax': 255, 'vmax': 255}


posList = []
while True :
    # Grab the image/video
    success, img = cap.read()
    
    try:
        # Find the colour of the ball HSL (for hue, saturation, lightness) 
        imgColor, mask=myColorFinder.update(img, hsvVals)
    except:
        break
        
    # Find the location of ball 
    imgContours, contours=cvzone.findContours(img, mask, minArea=500)
    
    if contours:
        #choose the biggest contours and its center
        posList.append(contours[0]['center'])
        
    for i, pos in enumerate(posList):
            cv2.circle(imgContours, pos, 5, (0, 255, 0), cv2.FILLED)
            
            #connecting the circles 
            if i == 0:
                cv2.line(imgContours, pos, pos, (0, 255, 0), 2)
            else:
                cv2.line(imgContours, pos, posList[i-1], (0, 255, 0), 2)
    
    img = cv2.resize(imgContours, (0, 0), fx=0.7, fy=0.7, interpolation = None)
   
    cv2.imshow('imgColor', imgContours)
    if cv2.waitKey(50) & 0XFF== 27:
        break 

# close all the windows after displaying the image
cv2.destroyAllWindows()

In [91]:
# Using polynomial regression, we will predict if the ball will go inside the basket or not

# Intialize the video 
cap = cv2.VideoCapture(FILENAME)

# Finding fps and no. of frames to calculate time per frame
fps = cap.get(cv2.CAP_PROP_FPS)
frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# Initializing parameters for speed estimation
# meterPerPixel = actual distance / no. of pixels between the distance
timeInSeconds = fps/frames
meterPerPixel = 3.5/ 500
estimatedSpeeds = []

# Create the color finder object :
myColorFinder = ColorFinder(False)
hsvVals ={'hmin': 8, 'smin': 96, 'vmin':115 , 'hmax': 14, 'smax': 255, 'vmax': 255}

# Declaring variable to get all point trajectory
A, B, C = 0, 0, 0
posListX = []
posListY = []
# 1300 is the maximum width of the image
xList = [item for item in range(0, 1300)]
prediction = False

while True :
    counter += 1
    # Grab the image/video
    success, img = cap.read()

    try:
        # Find the colour of the ball HSL (for hue, saturation, lightness) 
        imgColor, mask = myColorFinder.update(img, hsvVals)
    except:
        break

    # Find the location of ball 
    imgContours, contours = cvzone.findContours(img, mask, minArea=500)

    if contours:
        #choose the biggest contours and its center
        posListX.append(contours[0]['center'][0])
        posListY.append(contours[0]['center'][1])
    
    if posListX:
        # Find the coefficients for polynomial equation
        A, B, C = np.polyfit(posListX, posListY, 2)
        
        for i, (posX, posY) in enumerate(zip(posListX, posListY)):
            pos = (posX, posY)
            cv2.circle(imgContours, pos, 10, (0, 255, 0), cv2.FILLED)
            
            #connecting the circles 
            if i == 0:
                cv2.line(imgContours, pos, pos, (0, 255, 0), 5)
            else:
                cv2.line(imgContours, pos, (posListX[i-1], posListY[i-1]), (0, 255, 0), 2)
        
        # Find the line of quadratic equation and plot the points in the image
        for x in xList:
            # Y = Ax^2 + Bx + C
            y = int( (A*(x**2)) + (B*x) + C )
            cv2.circle(imgContours, (x, y), 2, (255, 0, 255), cv2.FILLED)
    
        # Prediction:
        # The width of basket hoop is 330 - 430
        # and it it at height of 590 from the bottom of the image
        # Now, for Y = 590, find the roots of the equation: Ax^2 + Bx = Y - C and check if X lies within specified range
        # Do the prediction for basket with less than 10 frames
        if len(posListX) < 10:
            a = A
            b = B
            c = C - 590

            try:
                x = int((-b - math.sqrt(b**2 - (4 * a * c))) / (2 * a))
            except:
                break
            prediction = 330 < x <430

        if prediction:
            cvzone.putTextRect(imgContours, "Basket", (50, 150), 
                                   scale=5, thickness=5, colorR=(0, 200, 0), offset=20)
        else:
            cvzone.putTextRect(imgContours, "No Basket", (50, 150), 
                                   scale=5, thickness=5, colorR=(0, 0, 200), offset=20)
    
        
        # Average Speed
        for i, (posX, posY) in enumerate(zip(posListX, posListY)):
            if i == 0:
                continue
            if posListX[i] < 800 :
                continue
            else:
                distance = math.sqrt((posListX[i] - posListX[i-1])**2 + (posListY[i] - posListY[i-1])**2)
                # print(distance)
                distanceInPixels = abs(distance)
                
                if distanceInPixels == 0:
                        continue
                        
                distanceInMeters = distanceInPixels * meterPerPixel
                estimatedSpeeds.append(distanceInMeters / timeInSeconds)
                
                avgSpeed = 0
                for speed in estimatedSpeeds:
                    avgSpeed += speed
                avgSpeed /= len(estimatedSpeeds)
                
                speedInfo = 'Average Speed: ' + '{0:.2f}'.format(avgSpeed) + 'm/sec'
                
                cvzone.putTextRect(imgContours, str(speedInfo), (50, 225), 
                                   scale=3, thickness=3, colorR=(200, 0, 0), offset=15)
    
    img = cv2.resize(imgContours, (0, 0), fx=0.7, fy=0.7, interpolation = None)

    cv2.imshow('imgColor', imgContours)
    if cv2.waitKey(100) & 0XFF== 27:
        break 

# close all the windows after displaying the image
cv2.destroyAllWindows()

C:\Users\rohan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3398: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\rohan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py:3398: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
